In [ ]:
from IPython.display import HTML, display

# <font color='green'>Balance of excitation and inhibition </font>
One of the properties of the neurons in the brain is that the spike patterns are highly ireegular. The question is how such irregular spiking arises. One of the prominent hypothesis suggests that high spike irregularity is a consequence of balance of excitation and inhibition. This is based on the argument that integration of incoming spiking activity will reduce the irregualrity of the input. That is, even if the neuron is driven by Poisson type spikes, the output spike irregularity will be smaller than the input.


## <font color='green'>Aims of the tutorial </font>
### Simulate a Integrate-and-fire neuron model with excitatory and inibitory inputs
### Understand the role of excitation and inhibition balance on the neuron output firing rate and spike patterns

## <font color='green'>Learnign outcomes</font>
### You should be able to write a simple code in NEST
### Understand the importance of excitation and inhbition in shaping the spike pattern irregularity

## <font color='green'>What will you do</font>
### Execute the Python code and display the results and describe your observations
### Write small code snippetes to perform certain simulations

## <font color='green'>Campbell's theorem</font>

$\mu_V  =  \lambda_{exc}\int EPSP(t)dt  -  \lambda_{inh}\int IPSP(t)dt$ \\

$\sigma^2_V =  \lambda_{exc}\int EPSP^2(t)dt +  \lambda_{inh}\int IPSP^2(t)dt $

$\lambda_i$ is the firing rate of the Poisson type spike trains coming from excitatory of inhibitory inputs
$EPSP(t)$ is the shape of the post-synaptic potential ilicited by an excitatory spike
$IPSP(t)$ is the shape of the post-synaptic potential ilicited by an inhbitory spike
$\mu_V$ is the mean of the membrane potential 
$\sigmaV$ is the standard deviation of the membrane potential 

The argument is that when a neuron receives both excitation and inhibition its membrane potential is reduced but the variance is increased resulting in more irregular firing rate. 

By contrast if the neuron only received excitatory inputs, its mean will go above the spike threshold and neuron will spike in a regular manner. And if the input is less than the spike threshold, even than due to low-pass filter nature of the neuron, the irregulatiry will be reduced. 

In this tutorial we will explore this effect of excitation and inhibition.


In [ ]:
import numpy as np
import matplotlib as matplt
%matplotlib inline
import pylab as plt
import nest
import time
np.random.seed(1000)


## <font color='green'>First lets see how a neuron reacts to synaptic input</font>

In [ ]:
# Create a neuron in NEST
# Lets create a single IAF neuron to which you can conect conductance-based synapses
# The shape of the PSC is determined by alpha function -- which is an integral of a exponential function
# When the weight is +ve we get excitatory synapse and 
# when the weight is -ve we get inhibitory synapse
nest.ResetKernel()
iaf_neuron = nest.Create('iaf_psc_alpha',1) 

# In case you want to change the To change the neuron parameters
neuron_params = {'V_th':-54., 
                 'V_reset': -70.0, 
                 't_ref': 2.0, 
                 'C_m':200.0, 
                 'E_L': -70.}
nest.SetStatus(iaf_neuron,neuron_params)



# we also create something called Parrot neurons to control the kind of inputs the neuron will get
parrot_ex = nest.Create('parrot_neuron',1)
parrot_in = nest.Create('parrot_neuron',1)

# Create spike generators and connect
gex = nest.Create('spike_generator', params = {'spike_times': [100.,200.,215.,230.]})
gin = nest.Create('spike_generator',  params = {'spike_times':[400.,500.,515.,530.]})

nest.Connect(gex,parrot_ex)
nest.Connect(gin,parrot_in)

# Create synapses
syn_param_exc = {'weight':20.,'delay':1.0}
syn_param_inh = {'weight':-20.5,'delay':1.0}

nest.CopyModel("static_synapse","syn_exc",syn_param_exc)
nest.CopyModel("static_synapse","syn_inh",syn_param_inh)

nest.Connect(parrot_ex, iaf_neuron, syn_spec={'model':'syn_exc'}) #4.5,1.) # excitatory
nest.Connect(parrot_in, iaf_neuron, syn_spec={'model':'syn_inh'}) #4.5,1.) # inhibitory

multimeter = nest.Create("multimeter")
nest.SetStatus(multimeter, {"withtime":True, "record_from":["V_m"]})

# and spike detector to the neuron
spikedetector = nest.Create("spike_detector",
                params={"withgid": True, "withtime": True})

# Finally connect them to the neuron
nest.Connect(multimeter, iaf_neuron)
nest.Connect(iaf_neuron, spikedetector)

nest.Simulate(800.)

### <font color='green'>Display the results</font>

In [ ]:
# And display the membrane potential
dmm = nest.GetStatus(multimeter)[0]
Vms = dmm["events"]["V_m"]
mem_ts = dmm["events"]["times"]

plt.figure(figsize=(20, 5))
plt.plot(mem_ts,Vms,linewidth=2,label='voltage')
#plt.ylim(-75,-50)
plt.xlim(0,1000)
plt.legend()
plt.ylabel('Voltage [mV]')
plt.xlabel('Time [msec]')
plt.legend()
plt.show()

## <font color='green'>IAF Neuron output driven by Poisson type excitatory spikes</font>

In [ ]:
# Create a neuron in NEST
# Drive an IAF neuron with Excitatory Poisson Input
# 
nest.ResetKernel()
iaf_neuron = nest.Create('iaf_psc_alpha',2) 

# In case you want to change the To change the neuron parameters
neuron_params = {'V_th':-54., 
                 'V_reset': -70.0, 
                 't_ref': 2.0, 
                 'C_m':200.0, 
                 'E_L': -70.}
nest.SetStatus(iaf_neuron,neuron_params)

# For one neuron we will set the spike threshold very high so that it does not spike
# This will allow us to record the Free Membrane Potential and we can relate it to the Campbell's theorem

nest.SetStatus([iaf_neuron[1]],{'V_th':1000.})

# we also create something called Parrot neurons to control the kind of inputs the neuron will get
parrot_ex = nest.Create('parrot_neuron',1)

# Poisson spike train
poi_exc = nest.Create('poisson_generator',1,{'rate':2000.})

# Connect the Poisson generator to the Parrot Neuron
# This will allow us to inject the same input to the two neurons
# One with spike and anther without spike

nest.Connect(poi_exc,parrot_ex)

# Create synapses
syn_param_exc = {'weight':30.,'delay':1.0}

nest.CopyModel("static_synapse","syn_exc",syn_param_exc)

# Connect the neuron
nest.Connect(parrot_ex, iaf_neuron, syn_spec={'model':'syn_exc'}) #4.5,1.) # excitatory

multimeter = nest.Create("multimeter")
nest.SetStatus(multimeter, {"withtime":True, "record_from":["V_m"]})

# and spike detector to the neuron
spikedetector = nest.Create("spike_detector",
                params={"withgid": True, "withtime": True})

# Finally connect them to the neuron
nest.Connect(multimeter, iaf_neuron)
nest.Connect(iaf_neuron, spikedetector)

# Simulate the neuron
nest.Simulate(1000.) # Simulation time in msec

In [ ]:
# And display the membrane potential
dmm = nest.GetStatus(multimeter)[0]
Vms = dmm["events"]["V_m"]
mem_ts = dmm["events"]["times"]
nid = dmm["events"]["senders"]

mem_s = Vms[nid==1] # spiking neuron
mem_ns = Vms[nid==2] # non-spiking neuron
mem_ts_s = mem_ts[nid==1]
mem_ts_ns = mem_ts[nid==2]

fmp_mean = np.mean(mem_ns)
fmp_var = np.var(mem_ns)

# And get the spike events
dSD = nest.GetStatus(spikedetector,keys="events")[0]
evs = dSD["senders"]
spk_ts = dSD["times"]

# Calculate the spike irregularity
isi = np.diff(spk_ts)
cv_isi = np.std(isi)/np.mean(isi)

print('CV-ISI = ' + str(cv_isi)) # Id CV_ISI is one it is a Poisson type spikinng. A regular spiking will result CV_ISI = 0
print ('Mean of Free Mem Pot. = ' + str(fmp_mean))
print ('Variance of Free Mem Pot. = ' + str(fmp_var))
print ('Spike rate = ' + str(len(spk_ts)))

plt.figure(figsize=(20, 5))
plt.plot(mem_ts_s,mem_s,linewidth=2,label='spiking mem pot.')
plt.plot(mem_ts_ns,mem_ns,linewidth=2,label='free mem pot.')
plt.plot(spk_ts, np.ones(len(spk_ts))*-54., ".",ms=20)
plt.ylim(-85,-30)
plt.xlim(0,1000)
plt.legend()
plt.ylabel('Voltage [mV]')
plt.xlabel('Time [msec]')
plt.legend()
plt.show()
#plt.show()

# <font color=salmon>TO DO</font>
## Vary the input rate and spike rate of the Poisson generator 
### - see how much you can increase the CV-ISI.
### - study how the increase in the spike rate and synapse weigh affects the mean and variance of the free membrane potential

## <font color='green'>Now lets inject excitation and inhibition both</font>

In [ ]:
# Create a neuron in NEST
# Drive an IAF neuron with Excitatory Poisson Input
# 
nest.ResetKernel()
iaf_neuron = nest.Create('iaf_psc_alpha',2) 

# In case you want to change the To change the neuron parameters
neuron_params = {'V_th':-54., 
                 'V_reset': -70.0, 
                 't_ref': 2.0, 
                 'C_m':200.0, 
                 'E_L': -70.}
nest.SetStatus(iaf_neuron,neuron_params)

# For one neuron we will set the spike threshold very high so that it does not spike
# This will allow us to record the Free Membrane Potential and we can relate it to the Campbell's theorem

nest.SetStatus([iaf_neuron[1]],{'V_th':1000.})

# we also create something called Parrot neurons to control the kind of inputs the neuron will get
parrot_ex = nest.Create('parrot_neuron',1)
parrot_in = nest.Create('parrot_neuron',1)

# Poisson spike train
poi_exc = nest.Create('poisson_generator',1,{'rate':42000.})
poi_inh = nest.Create('poisson_generator',1,{'rate':40000.})

# Connect the Poisson generator to the Parrot Neuron
# This will allow us to inject the same input to the two neurons
# One with spike and anther without spike

nest.Connect(poi_exc,parrot_ex)
nest.Connect(poi_inh,parrot_in)

# Create synapses
syn_param_exc = {'weight':30.,'delay':1.0}
syn_param_inh = {'weight':-30.,'delay':1.0}

nest.CopyModel("static_synapse","syn_exc",syn_param_exc)
nest.CopyModel("static_synapse","syn_inh",syn_param_inh)

# Connect the neuron
nest.Connect(parrot_ex, iaf_neuron, syn_spec={'model':'syn_exc'}) #4.5,1.) # excitatory
nest.Connect(parrot_in, iaf_neuron, syn_spec={'model':'syn_inh'}) #4.5,1.) # excitatory

multimeter = nest.Create("multimeter")
nest.SetStatus(multimeter, {"withtime":True, "record_from":["V_m"]})

# and spike detector to the neuron
spikedetector = nest.Create("spike_detector",
                params={"withgid": True, "withtime": True})

# Finally connect them to the neuron
nest.Connect(multimeter, iaf_neuron)
nest.Connect(iaf_neuron, spikedetector)

# Simulate the neuron
nest.Simulate(1000.) # Simulation time in msec

In [ ]:
# And display the membrane potential
dmm = nest.GetStatus(multimeter)[0]
Vms = dmm["events"]["V_m"]
mem_ts = dmm["events"]["times"]
nid = dmm["events"]["senders"]

mem_s = Vms[nid==1] # spiking neuron
mem_ns = Vms[nid==2] # non-spiking neuron
mem_ts_s = mem_ts[nid==1]
mem_ts_ns = mem_ts[nid==2]

fmp_mean = np.mean(mem_ns)
fmp_var = np.var(mem_ns)

# And get the spike events
dSD = nest.GetStatus(spikedetector,keys="events")[0]
evs = dSD["senders"]
spk_ts = dSD["times"]

# Calculate the spike irregularity
isi = np.diff(spk_ts)
cv_isi = np.std(isi)/np.mean(isi)

print('CV-ISI = ' + str(cv_isi)) # Id CV_ISI is one it is a Poisson type spikinng. A regular spiking will result CV_ISI = 0
print ('Mean of Free Mem Pot. = ' + str(fmp_mean))
print ('Variance of Free Mem Pot. = ' + str(fmp_var))
print ('Spike rate = ' + str(len(spk_ts)))

plt.figure(figsize=(20, 5))
plt.plot(mem_ts_s,mem_s,linewidth=2,label='spiking mem pot.')
plt.plot(mem_ts_ns,mem_ns,linewidth=2,label='free mem pot.')
plt.plot(spk_ts, np.ones(len(spk_ts))*-54., ".",ms=20)
plt.ylim(-85,-30)
plt.xlim(0,1000)
plt.legend()
plt.ylabel('Voltage [mV]')
plt.xlabel('Time [msec]')
plt.legend()
plt.show()
#plt.show()

# <font color=blue>Describe</font>
### How does the neuron output rate and spike variability changes with introduction of inhibition together with excitation. 

# <font color=salmon>TO DO</font>
## Write a code to generate the spike rate, mean and variance of the mem pot. as a function of excitatory adn inhibitory input rate. This way you can estimate the two-dimensional transfer function of a neuron

# <font color=salmon>TO DO</font>
## Summarize your observations and conclusions.